In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


/home/mameuio/miniconda3/LLM/zhiweidoushu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 环境与向量检索配置
load_dotenv()

True

In [20]:
# LLM 与提示模板
from openai import OpenAI
client = OpenAI()

generate_query_prompt_tmp = """
角色: 你是一个专业的紫微斗数问题生成器，用于 RAG(检索增强生成)。
目标: 针对输入的紫微斗数中文文本段，生成若干高质量、可用于检索的提问，提炼该段的核心宫位、星曜组合及格局，并按相关性排序。
约束:
- 仅依赖文本中的信息，不要杜撰未出现的宫位、主星、辅星、煞星或格局。
- 若文本涉及四化（禄、权、科、忌），至少生成一条涉及四化的提问；如未涉及，不必强加。
- 使用标准中文术语，短语之间用空格分隔；每条问题精炼但信息完整；生成 1-3 条，按相关性从高到低排序。
- 仅输出 JSON 字符串数组，不含解释、标题或代码块标记。
步骤:
1) 阅读文本，识别其中涉及的宫位、主星、副星、煞星、四化及三方四正关系。
2) 根据内容设计针对性问题，涵盖核心星曜组合及其对事业、感情、健康、人际等主题的影响。
3) 去重并按相关性降序输出问题。
示例输出: ["紫微破军在命宫？", "夫妻宫遇擎羊"]
输入: {text}
输出: ["Q1","Q2","Q3"]
"""
def generate_questions(doc):
    prompt = generate_query_prompt_tmp.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
# import os, json
# from langchain.schema import Document

# import hashlib
# stored_parent_docs = []

# for fname in os.listdir("./store_location"):
#     with open(os.path.join("./store_location", fname), "r", encoding="utf-8") as f:
#         doc_tmp = json.load(f)
        
#         header_tmp = str(doc_tmp.get("kwargs", {}).get("metadata", {}).get("Header 1", "")).strip()
#         text_tmp = str(doc_tmp.get("kwargs", {}).get("page_content", ""))
#         hash_object = hashlib.md5(header_tmp.encode())
#         hash_hex = hash_object.hexdigest()
#         id_tmp =  hash_hex[:8]
#         saved_doc = {"id":id_tmp,"text":text_tmp}
#         stored_parent_docs.append(saved_doc)

# #save this documents result
# import json
# with open('documents-with-ids.json', 'wt') as f_out:
#     json.dump(stored_parent_docs, f_out, indent=2)

In [13]:
import json
with open('documents-with-ids.json','r') as f_in:
    docs_with_id = json.load(f_in)

In [23]:
from tqdm.auto import tqdm
results = {}
for doc in tqdm(docs_with_id): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████| 228/228 [08:18<00:00,  2.18s/it]


In [ ]:
import pickle
#save the questions
with open('results.bin', 'wb') as f_out:
    pickle.dump(results,f_out)

In [32]:
# read the questions
with open('results.bin', 'rb') as f_in:
    results2 = pickle.load(f_in)

In [43]:
parsed_result = {}
for doc_id, json_questions in results2.items():
    parsed_result[doc_id] = json.loads(json_questions)

In [49]:
doc_index = {d['id']: d for d in docs_with_id}


In [ ]:
final_results = []

for doc_id, questions in parsed_result.items():
    for q in questions:
        final_results.append((q, doc_id))



In [57]:
import pandas as pd
df = pd.DataFrame(final_results, columns=['question', 'document'])
df.to_csv('ground-truth-data.csv', index=False)
